In [1]:

import subprocess
import sys
 


def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

 
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "numpy", "torch"])
 
install("pandas")
install("scikit-learn")
install("matplotlib")
install("xgboost")
install("statsmodels")
install("cvxpy")
install("Mapie")

  Using cached numpy-2.1.1-cp312-cp312-macosx_14_0_x86_64.whl.metadata (60 kB)
Using cached numpy-2.1.1-cp312-cp312-macosx_14_0_x86_64.whl (6.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.11.4 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.1.1 which is incompatible.


  Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl (20.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-quantile 0.1.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.


In [20]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import random
from data_analysis.data_analysis_utils import *


from data_analysis.datasets.datasets import GetDataset


from VennCalibration.VennQuantilePredictor import *
from VennCalibration.quantile_regression import *
from VennCalibration.CQR import *


def prepare_data(data_name, base_path, random_state=3, p_train=0.5, p_cal=0.3, log_transform_y = False):
    """
    Load data, preprocess features, and split into train, calibration, and test sets.

    Parameters:
    - filepath (str): Path to the CSV file containing the data.
    - random_state (int): Seed used by the random number generator for reproducibility.
    - p_train (float): Proportion of data to be used for training.
    - p_cal (float): Proportion of data to be used for calibration.
    - p_test (float): Proportion of data to be used for testing.
    - response_column (str): Name of the column to be used as the response variable.

    Returns:
    - X_train (np.ndarray): Features for the training set.
    - y_train (np.ndarray): Response for the training set.
    - X_cal (np.ndarray): Features for the calibration set.
    - y_cal (np.ndarray): Response for the calibration set.
    - X_test (np.ndarray): Features for the test set.
    - y_test (np.ndarray): Response for the test set.
    """

    p_test= 1 - p_train -p_cal
    # Load data
    X,y = GetDataset(data_name , base_path)
    y = np.array(y).reshape(-1)


    if log_transform_y:
      y = np.log(1 + y)

    # Split data into train, calibration, and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - p_train, random_state=random_state)
    X_cal, X_test, y_cal, y_test = train_test_split(X_test, y_test, test_size=p_test / (p_cal + p_test), random_state=random_state)

    return X_train, y_train, X_cal, y_cal, X_test, y_test





def run_conformal_analysis(data_name, base_path, random_state=10, p_train=0.5, p_cal=0.3, alpha = 0.1, log_transform_y = False):

    # Import external utilities (make sure this module is available in your path)



    X_train, y_train, X_cal, y_cal, X_test, y_test = prepare_data(data_name, 
                                                                  base_path,
        random_state=random_state,
        p_train=p_train, p_cal=p_cal,
        log_transform_y=log_transform_y)



    # venn prediction
    predictor_venn, VQP = conformal_venn_prediction(X_train, y_train, X_cal, y_cal, alpha = alpha, return_model = True)
    y_median_venn, interval_venn, threshold_venn = predictor_venn(X_test)
    quantile_uncalibrated = VQP.predict_score_quantile(X_test, calibrate=False)
    quantile_uncalibrated_cal = VQP.predict_score_quantile(X_cal, calibrate=False)

    interval_uncalibrated = np.vstack([y_median_venn - quantile_uncalibrated, y_median_venn + quantile_uncalibrated]).T

    y_median_cal, _, _ = predictor_venn(X_cal)
    

    def CQR_symmetric(quantile_uncalibrated_cal, y_median_cal, y_cal, alpha):
        n = len(y_cal)  # Number of calibration samples
        # Adjust alpha for finite sample size
        print(alpha)
        alpha_adjusted = alpha * (1 + 1 / n)
        # Compute the adjustment term
        adjustments = np.abs(y_median_cal - y_cal) - quantile_uncalibrated_cal
        adjustment_term = np.quantile(adjustments, 1 - alpha_adjusted)
        print(np.mean(quantile_uncalibrated_cal + adjustment_term <= np.abs(y_median_cal - y_cal)))
        return adjustment_term
    adjustment_term = CQR_symmetric(quantile_uncalibrated_cal, y_median_cal, y_cal, alpha)
    interval_cqr_symmetric = np.vstack([y_median_venn - quantile_uncalibrated - adjustment_term, y_median_venn + quantile_uncalibrated + adjustment_term]).T
 

    # Mondrian venn prediction with 10 bins
    predictor_mondrian = conformal_venn_prediction(X_train, y_train, X_cal, y_cal, alpha = alpha, quantile_calibrator = quantile_calibrator_histogram, calibrator_params = {"num_bin" : 5})
    _, interval_mondrian_5, _ = predictor_mondrian(X_test)

    predictor_mondrian = conformal_venn_prediction(X_train, y_train, X_cal, y_cal, alpha = alpha, quantile_calibrator = quantile_calibrator_histogram, calibrator_params = {"num_bin" : 10})
    _, interval_mondrian_10, _ = predictor_mondrian(X_test)
    
    # CQR baseline
    predictor_cqr = CQR(X_train, y_train, X_cal, y_cal, alpha = alpha)
    _, interval_cqr = predictor_cqr(X_test) 


    # Compute unconditional CP
    predictor_marginal = conformal_venn_prediction(X_train, y_train, X_cal, y_cal, alpha = alpha, quantile_calibrator = quantile_calibrator_histogram, calibrator_params = {"num_bin" : 1})
    _, intervals_uncond, _ = predictor_marginal(X_test)

 
    results = {
        "Method": [],
        "Coverage (marginal)": [],
        "Average Width": [],
        "Calibration Error": [],
        "coverage_by_bin": [],
        "weights_by_bin": []
    }
    names = ["Uncalibrated", "Venn-Abers",  "Marginal", "CQR (symmetric)", "CQR", "Venn-Mondrian (5 bins)", "Venn-Mondrian (10 bins)"]
    methods = [interval_uncalibrated, interval_venn, intervals_uncond, interval_cqr_symmetric, interval_cqr, interval_mondrian_5, interval_mondrian_10]

    for method, name in zip(methods, names):

        # marginal coverage
        marginal_coverage, width = compute_coverage(method, y_test, boolean=None)
        
        # coverage by bin
        coverage_info_by_bin = calculate_coverage_in_bins(threshold_venn, method, y_test)
        bin_ids = np.array([item[0] for item in coverage_info_by_bin]).reshape(-1)
        weights_by_bin = np.mean([np.mean(threshold_venn == bin_id)  for bin_id in bin_ids])
        weights_by_bin = weights_by_bin / np.mean(weights_by_bin)
        coverage_by_bin = np.array([item[2] for item in coverage_info_by_bin]).reshape(-1)
        calibration_error = np.mean(weights_by_bin * (np.maximum((1 - alpha) - coverage_by_bin, 0) )) 
        
        results["Method"].append(name)
        results["Coverage (marginal)"].append(np.mean(marginal_coverage))
        results["Average Width"].append(np.mean(width))
        results["Calibration Error"].append(np.mean(calibration_error))
        results["coverage_by_bin"].append(coverage_info_by_bin)
        results["weights_by_bin"].append(weights_by_bin)

    df = pd.DataFrame(results)
    df = df.map(lambda x: float(f"{x:.3g}") if isinstance(x, (int, float)) else x)
    df = df.map(lambda x: f'{x:g}' if isinstance(x, (int, float)) else x)

    return df
    
    # Output the table in LaTeX format
     
    #print(df.to_latex(index=False))

    
 

In [21]:
path = "data_analysis/datasets/"
data_name = "community"

import numpy as np
import pandas as pd

def run_dataset(data_name):
    log_transform_y = (data_name == "meps_21")
 
    seed = 42
    np.random.seed(seed)
    num_reps = 100
    random_states = np.random.randint(0, 1e6, size=num_reps).tolist()
    # average over 10 data splits
    dfs = [run_conformal_analysis(data_name, path, log_transform_y = log_transform_y, random_state = state) for state in random_states]


    # Combine dataframes with iteration metadata
    combined_df = pd.concat(
        [df.assign(iter=i + 1) for i, df in enumerate(dfs)], ignore_index=True
    )

    methods = ["Uncalibrated", "Venn-Abers",  "Marginal", "CQR (symmetric)", "CQR", "Venn-Mondrian (5 bins)", "Venn-Mondrian (10 bins)"]

    # Create a list to store results
    results = []

    for method in methods:
        df_sub = combined_df[combined_df['Method'] == method]
        marginal_coverage = np.nanmean(pd.to_numeric(df_sub['Coverage (marginal)'], errors='coerce').values)
        marginal_width = np.nanmean(pd.to_numeric(df_sub['Average Width'], errors='coerce').values)
        conditional_calibration_error = np.nanmean(pd.to_numeric(df_sub['Calibration Error'], errors='coerce').values)
    
        # Append the results to the list
        results.append({
            "Method": method,
            "Marginal Coverage": marginal_coverage,
            "Average Width": marginal_width,
            "Conditional Calibration Error": conditional_calibration_error
        })

    # Convert the results list into a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

results = [run_dataset(data_name) for data_name in ['bike', 'bio', 'star', 'meps_21', 'concrete', 'community']]

0.1
0.1001531393568147


/Users/larsvanderlaan/repos/VennCalibration/myenv2/lib/python3.13/site-packages/mapie/utils.py:620: UserWarning: WARNING: The predictions are ill-sorted.
  warnings.warn(
/Users/larsvanderlaan/repos/VennCalibration/myenv2/lib/python3.13/site-packages/mapie/utils.py:620: UserWarning: WARNING: The predictions are ill-sorted.
  warnings.warn(


0.1
0.1001531393568147


/Users/larsvanderlaan/repos/VennCalibration/myenv2/lib/python3.13/site-packages/mapie/utils.py:620: UserWarning: WARNING: The predictions are ill-sorted.
  warnings.warn(
/Users/larsvanderlaan/repos/VennCalibration/myenv2/lib/python3.13/site-packages/mapie/utils.py:620: UserWarning: WARNING: The predictions are ill-sorted.
  warnings.warn(


0.1
0.1001531393568147


...

/Users/larsvanderlaan/repos/VennCalibration/myenv2/lib/python3.13/site-packages/mapie/utils.py:620: UserWarning: WARNING: The predictions are ill-sorted.
  warnings.warn(
/Users/larsvanderlaan/repos/VennCalibration/myenv2/lib/python3.13/site-packages/mapie/utils.py:620: UserWarning: WARNING: The predictions are ill-sorted.
  warnings.warn(
/Users/larsvanderlaan/repos/VennCalibration/data_analysis/datasets/datasets.py:335: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  attrib = pd.read_csv(base_path + 'communities_attributes.csv', delim_whitespace = True)


0.1
0.10033444816053512


/Users/larsvanderlaan/repos/VennCalibration/myenv2/lib/python3.13/site-packages/mapie/utils.py:620: UserWarning: WARNING: The predictions are ill-sorted.
  warnings.warn(
/Users/larsvanderlaan/repos/VennCalibration/data_analysis/datasets/datasets.py:335: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  attrib = pd.read_csv(base_path + 'communities_attributes.csv', delim_whitespace = True)


0.1
0.10033444816053512


/Users/larsvanderlaan/repos/VennCalibration/myenv2/lib/python3.13/site-packages/mapie/utils.py:620: UserWarning: WARNING: The predictions are ill-sorted.
  warnings.warn(
/Users/larsvanderlaan/repos/VennCalibration/myenv2/lib/python3.13/site-packages/mapie/utils.py:620: UserWarning: WARNING: The predictions are ill-sorted.
  warnings.warn(


In [23]:
datasets = ['bike', 'bio', 'star', 'meps_21', 'concrete', 'community']


all_results = pd.concat(
    [result.assign(dataset=data_name) for result, data_name in zip(results, datasets)],
    ignore_index=True
)

latex_code = all_results.to_latex(index=False)

# Print or save the LaTeX code
print(latex_code)
all_results

\begin{tabular}{lrrrl}
\toprule
Method & Marginal Coverage & Average Width & Conditional Calibration Error & dataset \\
\midrule
Uncalibrated & 0.812940 & 83.069000 & 0.106437 & bike \\
Venn-Abers & 0.896600 & 104.159000 & 0.019467 & bike \\
Marginal & 0.898150 & 139.560000 & 0.100827 & bike \\
CQR (symmetric) & 0.898870 & 98.318000 & 0.030863 & bike \\
CQR & 0.901090 & 130.510000 & 0.021049 & bike \\
Venn-Mondrian (5 bins) & 0.901390 & 99.048000 & 0.032657 & bike \\
Venn-Mondrian (10 bins) & 0.900480 & 102.728000 & 0.022104 & bike \\
Uncalibrated & 0.848590 & 12.101000 & 0.088233 & bio \\
Venn-Abers & 0.898360 & 14.186000 & 0.016546 & bio \\
Marginal & 0.899160 & 14.865000 & 0.052835 & bio \\
CQR (symmetric) & 0.899750 & 13.915000 & 0.020485 & bio \\
CQR & 0.899770 & 13.894000 & 0.018678 & bio \\
Venn-Mondrian (5 bins) & 0.900350 & 14.165000 & 0.024948 & bio \\
Venn-Mondrian (10 bins) & 0.900750 & 14.428000 & 0.020175 & bio \\
Uncalibrated & 0.798780 & 615.540000 & 0.108731 & star \\


,Method,Marginal Coverage,Average Width,Conditional Calibration Error,dataset
0,Uncalibrated,0.81294,83.06900,0.106437,bike
1,Venn-Abers,0.89660,104.15900,0.019467,bike
2,Marginal,0.89815,139.56000,0.100827,bike
3,CQR (symmetric),0.89887,98.31800,0.030863,bike
4,CQR,0.90109,130.51000,0.021049,bike
5,Venn-Mondrian (5 bins),0.90139,99.04800,0.032657,bike
6,Venn-Mondrian (10 bins),0.90048,102.72800,0.022104,bike
7,Uncalibrated,0.84859,12.10100,0.088233,bio
8,Venn-Abers,0.89836,14.18600,0.016546,bio
9,Marginal,0.89916,14.86500,0.052835,bio
